# <font color='#d35400'> Lab 7 | Dimensionality Reduction & Unsupervised Machine Learning </font>
Welcome to Lab 7! This lab analyzes 63,542 emails. We convert these emails into raw text using a feature matrix of a "bag of words" model. Each column of the feature matrix corresponds to one word, each row corresponds to one email, and the entry stores the number of times that word was found in the email. 

<p align="center">
  <img src="coffee_dog.jpg" alt="Alt Text", width="300"/>
</p>

## <font color = '#FF8C00'> Section 1 </font> | Data Loading
In this section, we load a dataset of emails that are stored as individual JSON files within a directory called "email json". Each file follows the naming format "message XXXXX.json". The goal is to write a python script that locates and loads all JSON files, parsing them into a list of dictionaries. The list is then converted into a Pandas Dataframe.

- [x] Extract the provided email json.zip ﬁle. It should create a directory called “email json”. Each email is stored as a separate JSON document with a name in the format “message XXXXX.json”.
- [x] Remember when I introduced the Titanic data set in the ﬁrst lab and I asked the class to import the ﬁle by hand? That practice will pay oﬀ here. Write some code to ﬁnd and load all of the JSON documents. You should have a list of dicts when done (Hint: Review the built-in Python json library).
- [x] Convert the list of dicts into a Pandas DataFrame. The DataFrame should have 5 columns and 63,452 rows (Hint: use the DataFrame.from records() or DataFrame.from dict() functions).

### <font color = '#FF8C00'> Extracting the Zip File </font>
We start off by extracting the zip file. We then store the `.json` files in a directory named `email_json`. We know that the JSON files come with a name in the format `messageXXXXX.json`.

In [ ]:
# importing the os library
import os

# listing out the files in the directory
directory = r"C:\GitHub\DataScienceMachineLearning\wk_09\lab\email_json"
files = os.listdir(directory)
print(files[0])

### <font color = '#FF8C00'> Loading the JSON Documents </font>
Next, we load all the JSON documents and produce a list of dicts using the Python JSON library. We achieve this by going into every JSON document and parsing them into a dictionary.

In [ ]:
# importing the json library
import json

# importing the tqdm library
from tqdm import tqdm

# setting up the dictionary
email_dictionary = []

for filename in tqdm(files):
    file_path = os.path.join("email_json", filename)
    with open(file_path, 'r') as file:
        email_contents = json.load(file)
        email_dictionary.append(email_contents)

### <font color = '#FF8C00'> Creating a Pandas Dictionary </font>
We now convert the list of dictionaries into a pandas data frame. The data frame is expected to have 5 columns and 63,452 rows. We achieve this using `DataFrame.fromrecords()` and `DataFrame.fromdict()`.

In [ ]:
# importing the pandas library
import pandas as pd

# importing the ipython display
from IPython.display import display

# converting the dictionary into a data frame
email_df = pd.DataFrame.from_records(email_dictionary)

# confirming the shape of the data frame
display(email_df.shape)


## <font color = '#FF8C00'> Section 2 </font> | Extracting the Features
In this section, we prepare the email message bodies for analysis. We do this by converting the raw text into a numerical format using a feature matrix. To achieve this, we use `Scikit-learn`'s `CountVectorizer` which records only the presence or absence of a word.

- [x] Use Scikit Learn’s CountVectorizer class with the binary=True ﬂag to create a feature matrix from the message bodies. When doing this set min df=10 to exclude any words that do not appear in at least 10 emails. The “bodies” column of the DataFrame can be used as a list of strings and passed directly into the ﬁt transform() method of the
CountVectorizer.


### <font color = '#FF8C00'> Preparation for Analysis </font>
Here, we extract the email message bodies to process the text data and convert it into a numerical format. What we end up getting is a matrix of 1's and 0's, where each row represents an email and each column represents a word.

In [ ]:
# importing count vectorizer
from sklearn.feature_extraction.text import CountVectorizer

# creating a count vectorizer object
count_vectorizer = CountVectorizer(binary=True, min_df=10)

# using fit_transform to create a feature matrix
feature_matrix = count_vectorizer.fit_transform(email_df['body'])

### <font color = '#FF8C00'> Visualizing Feature Matrix </font>
Lastly, we would like to check out the feature matrix. We do this by converting it into a Pandas data frame. This just allows us to visualize and check if what we're doing is right.

In [ ]:
# visualizing the feature matrix
features = count_vectorizer.get_feature_names_out()
feature_df = pd.DataFrame(feature_matrix.toarray(), columns=features)
display(feature_df)

## <font color = '#FF8C00'> Section 3 </font> | Dimensionality Reduction / Visualization
We now focus on reduce the dimensionality of the large feature matrix, we use `TruncatedSVD` from Scikit-learn with the `fit_transform()` function to create a new matrix with 10 components. We then plot the explained variance ratios of these components and create a scatter plot using the two components and a second scatter plot where the points are colored based on the category.

- [x] This matrix has too much information to be directly useful. Use the ﬁt transform() function of Scikit Learn’s TruncatedSVD class to transform the original feature matrix into a new feature matrix with 10 columns (variables or components). Scikit Learn’s TruncatedSVD method is similar to its PCA method, but it works with sparse matrices.
- [x] Plot the explained variance ratios of the components (Hint: use the explained variance ratio property of the TruncatedSVD class)
- [x] Create a scatter plot using the two components with the highest explained variance ratios (Hint: plt.scatter(proj matrix[:, i], proj matrix[:, j])).
- [x] Create a second scatter plot using the same two components. This time, color the points based on the category column of the DataFrame. All spam messages should be one color; all ham messages should be a second color.

### <font color = '#FF8C00'> Converting the Feature Matrix </font>
The current matrix we have has too much information to be useful. We use `fit_transform()` that comes from Scikit Learn's `TruncatedSVD` class to transform the original feature matrix into a new one with 10 columns.

In [ ]:
# importing the TruncatedSVD library
from sklearn.decomposition import TruncatedSVD

# initializing the TruncatedSVD object
svd = TruncatedSVD(n_components=10)

# applying fit_transform to reduce the feature matrix to 10 components
svd_10 = svd.fit_transform(feature_matrix)

# printing out the new shape
display(svd_10.shape)

### <font color = '#FF8C00'> Plotting Explained Variance </font>
Next, we plot the explained variance ratios of the components. We use the `explained_variance_ratio_property` of the TruncatedSVD class. 

In [ ]:
# importing the matplotlib library
import matplotlib.pyplot as plt

# plotting the plot using explained variance
plt.plot(svd.explained_variance_ratio_)
plt.title("Explained Variance Ratio by Component")

# labeling the x and y axes
plt.xlabel("Component")
plt.ylabel("Explained Variance Ratio")

# setting the limits from 0 to 10
plt.xlim(0, 10)

# saving the plot
plt.savefig(r'C:\GitHub\DataScienceMachineLearning\wk_09\plots\explainedvarianceratio.png', dpi=300)

# graphing out the plot
plt.show()

### <font color = '#FF8C00'> Plotting the First Scatter Plot </font>
Next, we create a scatter plot using the two components with the highest explained variance ratios. We use example code such as `plt.scatter(proj_matrix[:,i], proj_matrix[:,j])`.


In [ ]:
# importing numpy
import numpy as np

# finding the highest explained variance ratios
top_two = np.argsort(svd.explained_variance_ratio_)[-2:][::-1]

# extracting the highest values
i, j = top_two[0],top_two[1]

# plotting the scatter plot using the two components
plt.scatter(svd_10[:, i], svd_10[:,j])

# setting the title, x any y axis labels
plt.title("Scatter Plot of Two Components with Highest Explained Variance Ratios")
plt.xlabel("Component 1")
plt.ylabel("Component 2")

# saving the plot
plt.savefig(r'C:\GitHub\DataScienceMachineLearning\wk_09\plots\scatterplot.png', dpi=300)

# displaying the plot
plt.show()

### <font color = '#FF8C00'> Plotting the Second Scatter Plot </font>
Finally, we create a second scatter plot using the same two components. We color the points based on the category column of the DataFrame. The spam messages should be one color; the ham messages should be a second color.

In [ ]:
# setting up the coloring 
coloring = email_df['category'].map({'spam':'lightcoral', 'ham':'lightskyblue'})

# plotting the scatter plot using the two components
plt.scatter(svd_10[:, i], svd_10[:,j], c=coloring, marker='o')

# setting the title, x any y axis labels
plt.title("Scatter Plot of Two Components with Highest Explained Variance Ratios")
plt.xlabel("Component 1")
plt.ylabel("Component 2")

# manually creating legend and and labels for spam
spam_handle = plt.Line2D([0], [0], marker='o', color='w', label='Spam',
                          markerfacecolor='lightcoral', markersize=10)

# manually creating legend and and labels for ham
ham_handle = plt.Line2D([0], [0], marker='o', color='w', label='Ham', 
                        markerfacecolor='lightskyblue', markersize=10)

# setting up the legend
plt.legend(handles=[spam_handle, ham_handle])

# saving the plot
plt.savefig(r'C:\GitHub\DataScienceMachineLearning\wk_09\plots\scatterplotcolored.png', dpi=300)

# displaying the plot
plt.show()

## <font color = '#FF8C00'> Section 4 </font> | The Clustering Process
In this section, we perform clustering of emails based on their features, using a suitable clustering algorithm from Scikit-learn. We perform the necessary transformations, such as dimensionality reduction and we cluster the emails based on the top two components with the highest explained variance. The resulting clusters will be visualized in a scatter plot, with points colored by the cluster labels. 

- [x] To perform further analysis, we want to cluster the emails. By clustering the emails, each message will be assigned a cluster id (e.g., 0, 1, 2, etc.). There are numerous clustering algorithms, each with strengths and weaknesses. We cannot cover them all in lecture and so begin this section by reviewing the clustering algorithms available in Scikit Learn. Choose a clustering algorithm that you think would be appropriate for
this data set.
- [x] Cluster the samples using the two SVD components that you determined to have the highest explained ratio AFTER performing any necessary operations to transform them. The clustering algorithm should return a 1D numpy array of cluster labels (e.g., 0, 1, 2, etc.) for each point.
- [ ] As in 3.4., create a scatter plot of the two SVD components that you determined to have the highest explained ratio. The clustering algorithm should label the points so that all points in the same cluster have the same cluster id (With two clusters, there should only be two cluster ids). Color the points according to their cluster labels. If the clustering does not look correct, try adjusting the parameters of the clustering algorithm you chose or choose a diﬀerent algorithm.
- [ ] Calculate a confusion matrix for the ham / spam labels versus the cluster labels.

## <font color = '#FF8C00'> Centroid Based and Aggloromerative Hierarchial Clustering </font>
We start off by clustering the emails, where each message will be assigned a cluster ID. We choose a clustering algorithm that would be the most appropriate for the data set. We experiment with both centroid based clustering first and then aggloromerative hierarchial clustering.

### <font color = '#FF8C00'> The Centroid Based Clustering Approach </font>
Starting off with the centroid based clustering approach, we cluster the approach using the two SVD components that we have previously determined. The clustering algorithm will then return a 1D numpy array of cluster labels. We will be using k-means, and before we use k-means, we use the silhouette score to determine the best k value.

In [ ]:
# importing the silhouette score library and KMeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# storing all the silhouette scores
silhouette_scores = []

# extracting the top two SVD components
top2 = svd_10[:, [i, j]] 

# for looping through the different k values to find the perfect one
for k_value in tqdm(range(2, 20)):
    kmeans = KMeans(n_clusters=k_value, random_state=42)
    kmeans.fit(top2)
    score = silhouette_score(top2, kmeans.labels_)
    silhouette_scores.append(score)

# printing out the silhouette scores
print(silhouette_scores)

In [ ]:
# plotting out the silhouette scores 
plt.plot(silhouette_scores)

# setting the title, x and y labels
plt.title("Silhouette Scores vs K Values")
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Score")

# setting the x axis to show only whole numbers
plt.xticks(range(2, 20))
plt.xlim(2, 19)

# saving the plot
plt.savefig(r'C:\GitHub\DataScienceMachineLearning\wk_09\plots\silhouettelineplot.png', dpi=300)

# displaying the plot
plt.show()

### <font color = '#FF8C00'> Retrieving the 1D Numpy Array </font>
After finding the best value for k, we now cluster the samples and then use the clustering algorithms to return a 1D numpy array of cluster labels for each point.

In [ ]:
# applying the kmeans clustering with k = 2
kmeans2 = KMeans(n_clusters=2, random_state=42)
kmeans2.fit(top2)

# retrieving the cluster labels
cluster_labels = kmeans2.labels_

# printing out the cluster labels
print(cluster_labels)

### <font color = '#FF8C00'> Plotting the Centroid Based Scatter Plot </font>
Now, we create a scatter plot of the two SVD components. The clustering algorithm labels the points so that all points in the same cluster have the same cluster ID. We color the points according to their cluster labels. 

In [ ]:
# plotting the scatter plot
plt.scatter(svd_10[:, i], svd_10[:,j], c=cluster_labels, marker='o')

# plotting the title, x and y labels
plt.title("Scatter Plot of Top Two SVD Components with k = 2")
plt.xlabel("Component 1")
plt.ylabel("Component 2")

# displaying the legend
plt.colorbar(label='Cluster')

# displaying the plot
plt.show()

### <font color = '#FF8C00'> The Aggloromerative Hierarchial Clustering Approach </font>
Starting off with the centroid based clustering approach, we cluster the approach using the two SVD components that we have previously determined. The clustering algorithm will then return a 1D numpy array of cluster labels. We will be using k-means, and before we use k-means, we use the silhouette score to determine the best k value.

In [ ]:
# importing the agglormerative clustering approach
from sklearn.cluster import AgglomerativeClustering

# building and fitting the clustering model
agglomerative_model = AgglomerativeClustering(n_clusters=2)
agglomerative_model.fit(top2)

# retrieving the cluster labels
agglomerative_labels = agglomerative_model.labels_

# printing out the labels
print(agglomerative_labels)
